# 5.3 Particle Filter for Robot Localization

When dealing with problems of a stochastic nature (meaning, the processes involved are random), there are two main approaches:

- The first approach is to work **analytically**, with an explicitly defined probability distribution (.e.g., a Gaussian one), and estimate the parameters of that distribution. This is what we did with the Extended Kalman Filter.

- The second one is to generate many **random samples**, and follow what happens to each of them as they are subjected to the randomness of the process. Crucially, each of these samples represents a single, determined value, rather than a probability distribution, and is thus easier to work with. Techniques using this second approach are often referred to as *Monte Carlo Methods*, and are very common in many disciplines. 

In this chapter, we will see a sample-based technique for probabilistic estimation, and apply it to the problem of robot localization. This technique goes by multiple names, such as *Monte Carlo Localization*, but we will refer to it with the generic term of **Particle Filter**. Specifically, we will:
- Get an intuition about what a particle is (section 5.3.1),
- Implement the building blocks of the filter (section 5.3.2), and
- Put them all together to perform localization (section 5.3.3).

## 5.3.1 But, what is a particle?

If we want to represent a probability distribution with a finite set of discrete values, we can do this by repeatedly sampling the distribution. In the context of Monte Carlo methods, that is precisely how we generate particles: by drawing samples from a distribution. 

What makes it a **particle**, rather than just a **sample**, is that a particle will later be transformed by *other* random samples. That is, a particle holds state, and that state can evolve over time. Let's see a toy example to solidify our understanding of this concept.


### Basic example

Let's say we have a random variable representing a position $[x,y]^T$, whose PDF is a 2D gaussian. We want to modify this position by moving a certain amount to the right, but the movement itself has some uncertainty, also described in this case by a gaussian. 

Now, imagine that you do not know how to propagate the uncertainty from these random variables to estimate the PDF of the result (even though, you of course do know!). How could we get a numerical approximation of this PDF, which we don't know how to calculate analytically? Well, with particles! Have a look:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import copy

def basic_particles_example():
    num_particles = 10000
    start_mu = np.vstack([1, 1])
    start_sigma = np.diag([1, 1])

    # generate the particles by sampling the initial distribution
    particles = stats.multivariate_normal(start_mu.flatten(), start_sigma).rvs(num_particles)
    plt.scatter(particles[:,0], particles[:,1], s=0.1, color='blue')
    print("initial distribution: mean=({:.2f}, {:.2f}), variances=({:.2f}, {:.2f})".format(
        np.mean(particles[:,0]), np.mean(particles[:,1]), 
        np.var(particles[:,0]), np.var(particles[:,1])))


    colors= ['red', 'magenta', 'green']
    for i in range(3):
        # apply to each particle a single, random movement
        movement_mu = np.vstack([10, 0])
        movement_sigma = np.diag([2, 1])
        movement = stats.multivariate_normal(movement_mu.flatten(), movement_sigma).rvs(num_particles)

        particles += movement

        # plot the result
        plt.scatter(particles[:,0], particles[:,1], s=0.1, color=colors[i])
        print("Distribution after movement {:d}: mean=({:.2f}, {:.2f}), variances=({:.2f}, {:.2f})".format(
            i+1,
            np.mean(particles[:,0]), np.mean(particles[:,1]), 
            np.var(particles[:,0]), np.var(particles[:,1])))
    

basic_particles_example()

Hey, that's pretty good! Turns out we can propagate uncertainty just by drawing many samples for each movement, and keeping track of where each of the particles ends up after moving.

However, while this is an easy way of propagating uncertainty, we have the same problem we have had in the past: the uncertainty always gets larger, so we end up not knowing where we are at all. To correct this, and bring the uncertainty back down, we need to use sensory data (**observations**) to evaluate how good each particle is. 

Basically, the question we want ask is: *if the robot was at the position represented by this particle, how well does my existing observation match what I would expect to see?* Once we know this, we could remove very bad particles, and focus on the areas where good particles were found. That, in essence, is what the Particle Filter algorithm is.

In the rest of this chapter, we will see an example of how to do this with a **laser scanner**, using the **likelihood field** model. 

### Setting things up: occupancy map and likelihood field

Let's create a map, and use it to generate a **likelihood field**. Don't worry about the implementation details of this part, the important thing is what we get at the end: an occupancy map, and a likelihood field generated from it.

In [ ]:
import cv2
import time
from IPython.display import display, clear_output

import sys
sys.path.append("..")
from utils.laser.laser2D import Laser2D
from utils.DrawRobot import DrawRobot
from utils.tcomp import tcomp
from utils.RemapRange import RemapRange
from utils.PlotEllipse import PlotEllipse
from laserUtils import LaserUtils

In [ ]:
class Map:
    def __init__(self, contours_map : np.ndarray, resolution : float):
        self.MIN_LIKELIHOOD = 0.1 # minimum value allowed in the likelihood field. Must be >0

        self.min_x, self.min_y = (np.nanmin(contours_map[0]), np.nanmin(contours_map[1]))
        self.max_x, self.max_y = (np.nanmax(contours_map[0]), np.nanmax(contours_map[1]))

        self.__margin_size = 1
        self.occupancy_img = np.full([int((self.max_y + self.__margin_size) / resolution), 
                       int((self.max_x + self.__margin_size) / resolution)],
                      255, np.uint8)
        
        self.width, self.height = self.occupancy_img.shape
        self.resolution = resolution

        # create dense image from contours
        #--------------------------------
        # split into multiple contours and arrange into a list of 1x2 vectors
        
        # split into a different array every time there is a nan
        indices = np.unique(np.where(np.isnan(contours_map))[1])
        contours = np.hsplit(contours_map, indices) 
        contours = [c[~np.isnan(c)].reshape(2, -1) for c in contours] # get rid of the actual nans, keeping the 2D shape

        # remove empty contours
        contours = [c for c in contours if c.size > 0]
        contours_formatted = [(c / resolution).astype('int32').T.reshape(-1, 1, 2) for c in contours]
        
        # draw the contours on the image
        cv2.polylines(self.occupancy_img, contours_formatted, False, (0,0,0))

        # calculate the likelihood field with a distance transform
        #--------------------------------
        self.likelihood_field = cv2.distanceTransform(self.occupancy_img, cv2.DIST_L2, 5)

        # calculate likelihood from the pixel distances
        SIGMA_HIT_M = 0.5 # set this to control how fast the likelihood decays as you get farther from the object
        sigma_hit_px = SIGMA_HIT_M / resolution

        self.likelihood_field = stats.norm(loc=0, scale=sigma_hit_px).pdf(self.likelihood_field)
        
        # remaping the values here to avoid having too-low values which will cause numerical issues upon repeated multiplication
        # the actual value here is therefore *proportional* to the likelihood, not the likelihood itself
        # but that's ok, because we will normalize the particle weights later :)
        self.likelihood_field = RemapRange(self.likelihood_field, 
                                           0, np.max(self.likelihood_field),
                                           self.MIN_LIKELIHOOD, 1)

    def sample_likelihood(self, point : np.ndarray):
        if point[0] < self.min_x or point[0] >= self.max_x or point[1] < self.min_y or point[1] >= self.max_y:
            return self.MIN_LIKELIHOOD
        indices = np.rint(np.flip(point / self.resolution, 0)).astype('int32')
        value = self.likelihood_field[tuple(indices)]
        return value
    
    def is_valid_position(self, point: np.ndarray):
        if point[0] < self.min_x or point[0] >= self.max_x or point[1] < self.min_y or point[1] >= self.max_y:
            return False
        indices = np.rint(np.flip(point / self.resolution, 0)).astype('int32')
        value = self.occupancy_img[tuple(indices)]
        return value[0] > 0


    # Visualization
    #-------------------------------------
    def show_occupancy(self):
        return self.__print_img(self.occupancy_img)

    def show_likelihood(self):
        return self.__print_img(self.likelihood_field)
    
    def __print_img(self, img):
        size_factor = 0.7
        fig, ax = plt.subplots(figsize=(self.max_x * size_factor, self.max_y * size_factor))
        img = cv2.flip(img, 0) # images are printed with the origin at the top, so flip vertically

        ax.imshow(img, cmap='gray', extent=[0, self.max_x + self.__margin_size, 0, self.max_y + self.__margin_size])
        plt.xlim([0, self.max_x+self.__margin_size])
        plt.ylim([0, self.max_y+self.__margin_size])
        
        # Title and axis labels
        plt.xlabel('X position (m)')
        plt.ylabel('Y position (m)')
        return fig, ax
    

# Create the map from the contours
virtual_map = LaserUtils.create_room_map()
image_resolution = 0.01
occupancy_map = Map(virtual_map, image_resolution)

That is a lot of code, but don't worry. The result of all that is just two maps:

- The occupancy map (with object surfaces only)

In [ ]:
occupancy_map.show_occupancy();

- And the likelihood field

In [ ]:
occupancy_map.show_likelihood();

The likelihood field expresses a concept that can be a bit abstract for each point in the environment (a pixel in the image): when we take an observation with our laser scanner, what is the probability that one of the points in the point cloud will fall here? The intuition that this map codifies is that, while the sensor might be unreliable due to noise in the measurement, the measured points should still be *close* to the actual surfaces of the obstacles. The further away from the objects, the less likely we are to observe anything there.

Formally, this can be expressed as $p(z^w_i | m)$, where $z^w_i$ is a point in the world-space point cloud $z^w$, and $m$ is the map. Crucially, this expression does not depend on the pose of the robot, $x$, so it is just a function of the geometry of the map and the reliability of the sensor. 

It should be pointed out that the likelihood field is a somewhat simplified model: it does not take into consideration the plausibility of each laser beam trajectory or from how many viewpoints each obstacle could be observed. Still, the image above is a reasonable enough approximation of $p(z^w | m)$, and gives good results in practice while being quite easy to generate.

Using the likelihood field, we can now answer our previous question: if the robot was *here*, then how well does my sensor observation fit the map?

## 5.3.2 Implementing the Particle Filter building blocks

We will now look at each of the steps that go into creating the Particle Filter algorithm, individually:

- Evaluating the particles.
- Using the evaluations to estimate the real pose.
- Propagating particles between iterations (sampling).
- Replacing bad particles with better ones (resampling).

Then, once we have collected all the pieces, we'll assemble the puzzle and get the full picture.

### **<span style="color:green"><b><i>ASSIGNMENT 1: Evaluating particles</i></b></span>**  

Let's implement a function that takes in a map $m$ and a point cloud $z^w$, which represents the endpoints of a scan captured by a 2D laser scanner expressed in world coordinates, and returns the likelihood $p(z^w|m)$. The calculation we have to do is this:

$$
    p(z^w | m) \propto \prod_i p(z^w_i | m)
$$

Remember that $p(z^w_i | m)$ is precisely what the likelihood field represents. The expression above has a proportional-to sign rather than an equal sign,  but do not worry. That simply means that there are some normalization terms involved that we are not going to calculate directly. Instead, we will do a normalization step later on, to make sure the probabilities add up to 1. 

To make things easier, we provide you with the function `LaserUtils.express_as_global_point_cloud(z, pose)`, which does something we already saw in the sensing chapter: takes the sensor reading expressed in local space and polar coordinates, and turns it into a global-space, cartesian point cloud (which is what you need to sample the likelihood field).

The map class also provides a useful function: `sample_likelihood()`, which returns the value of the likelihood field at a specific coordinate of the map.

In [ ]:
def evaluate_pointcloud(map : Map, pointcloud): 
    total_prob = 1
    
    for i in range(pointcloud.shape[1]):
        None # calculate the likelihood for this point and accumulate using the formula in the text of the assignment

    return total_prob

def evaluate_pose(map: Map, z, pose):
    world_pointcloud = None
    return evaluate_pointcloud(None)

Now check that your function is right!

In [ ]:
def test_evaluate_pointcloud():
    pose =  np.vstack([4,4,0])
    laser = Laser2D(np.pi, 0.1, 10, np.diag([0.01, 0.0]), pose)
    z = laser.take_observation(virtual_map)

    likelihood = evaluate_pose(occupancy_map, z, pose)
    print(f"Likelihood of point cloud: {likelihood}")
    
    # Plot
    fig, ax = occupancy_map.show_likelihood()
    DrawRobot(fig, ax, pose, linewidth=2)
    LaserUtils.plot_observations(z, pose, ax, draw_lines=False)

# Set a seed for random numbers
np.random.seed(42)

test_evaluate_pointcloud()

<span style="color:blue">Expected output:</span>

```
Likelihood of point cloud: 0.7653936384712728
```

<img src="images/likelihood_field_with_scan.PNG">

Ok, that gives us a number for how believable this point cloud is, but that's not terribly useful on its own. The point of all this is that we can now **compare** point clouds to each other, and see how likely each of them are. 

Let's remind ourselves why we want to do this. The idea is that, given a sensor reading $z_c$ (in cartesian coordinates), in order to get a global-space point cloud $z^w$ we need to know the pose of the robot $x$ so:

$$ z^w = x \oplus z_c $$

If we have multiple candidate poses (particles), we can use each of them to reproject the readings and get a point cloud. If we now evaluate those point clouds, that is equivalent to evaluating the poses themselves.

And that is what we want! We want to test many possible poses, and use that to estimate what the real pose is.

Let's take that next step, then. Given a sensor reading and few candidate poses (particles), evaluate each pose with your brand new ``evaluate_pointcloud()`` function. Think about why each pose receives the evaluation that it gets, given that you can also see the real pose that the measurement was taken from (as usual, in red). 

In [ ]:
def PlotParticle(fig, ax, pose, z, color):
    DrawRobot(fig, ax, pose, 0.01, linewidth=2, color=color)
    LaserUtils.plot_observations(z, pose, ax, draw_lines=False, point_color=color)

def test_particle_evaluation():
    true_pose =  np.vstack([4,4,0])
    laser = Laser2D(np.pi, 0.1, 10, np.diag([0.01, 0.0]), true_pose)
    z = laser.take_observation(virtual_map)

    pose_particle1 = np.vstack([4.2, 4.4, 0.1])
    pose_particle2 = np.vstack([4.01, 4.1, -0.1])
    pose_particle3 = np.vstack([4.4, 3.8, -0.2])

    # plot
    fig, ax = occupancy_map.show_likelihood()
    DrawRobot(fig,ax, true_pose, 0.01, linewidth=2, color="red")
    PlotParticle(fig, ax, pose_particle1, z, "orangered")
    PlotParticle(fig, ax, pose_particle2, z, "olive")
    PlotParticle(fig, ax, pose_particle3, z, "magenta")

    # evaluate the particles
    eval_1 = None
    eval_2 = None
    eval_3 = None

    #print evaluations
    print(f"Particle 1 (orange): {eval_1}")
    print(f"Particle 2 (green): {eval_2}")
    print(f"Particle 3 (magenta): {eval_3}")

test_particle_evaluation()

<span style="color:blue">Expected output:</span>

```
Particle 1 (orange): 9.697793288623095e-06
Particle 2 (green): 0.2699737678772269
Particle 3 (magenta): 1.2663658417295267e-06
```

<img src="images/likelihood_field_with_particle_evaluations.png">

### Pose estimation from particles

So, if we have a list of particles representing possible values for the pose of the robot, and we can evaluate how good each of them are, how do we turn that into a single estimation of the *real* pose?

Probability theory gives us the perfect tool for this: the **expected value**. This is the same as the mean of a probability distribution, and can be very easily computed as a weighted average, where the weights are the probabilities of each value:

$$
E[X] = \sum_x p(x) \cdot x
$$

Since we don't quite have a probability distribution, but instead a set of particles, our computation is instead:

$$
E[X] = \sum_i w_i \cdot x_i
$$

, where $w_i$ is the weight of particle $i$, and $x_i$ is the pose it represents. 

We can also calculate the uncertainty of our current particle distribution by using this expected value to get the covariance matrix associated with our estimated pose. The definition of the covariance between two variables $X$ and $Y$ is as follows:

$$
cov_{XY} = E[(X-E[X]) \cdot (Y-E[Y])]
$$

So, in our case:

$$
cov_{XY} = \sum_i w_i \cdot [(x_i-E[X]) \cdot (y_i-E[Y])]
$$

For example, this means that: 

$$
\sigma^2_x = \sum_i w_i \cdot (x_i-E[X])^2 
$$
$$
\sigma^2_y = \sum_i w_i \cdot (y_i-E[X])^2
$$

### **<span style="color:green"><b><i>ASSIGNMENT 2: Estimating the real pose from the particle evaluations</i></b></span>**  

Let's implement a function, `current_estimation()`, which receives a list of pre-evaluated particles and returns the expected value of the pose, along with the covariance of that estimation. Since averaging angles is a little tricky, and we don't expect you to be an expert on circular statistics, you just need to handle the *position* part of the pose.

You are provided with a `Particle` class, which holds the pose which a particle represents and the weight it was assigned by the likelihood evaluation in the previous step. There is also an already implemented `normalize_weights()` function, which makes sure all the weights add up to 1. 

In [ ]:
class Particle:
    def __init__(self, pose, weight):
        self.pose = pose
        self.weight = weight

# This just makes it so the sum of all the weights is equal to 1
def normalize_weigths(particles):
    total_weight = 0.
    for particle in particles:
        total_weight += particle.weight

    for particle in particles:
        particle.weight /= total_weight

In [ ]:
def current_estimation(particles):
    normalize_weigths(particles)
    
    # we cannot just average angles, so instead we separate position and heading direction
    # averaging the (vector) heading directions is the most common definition of "circular mean" (https://en.wikipedia.org/wiki/Circular_mean)
    expected_value_position = np.vstack([0., 0.])
    expected_value_direction = np.vstack([0., 0.])

    # calculate the expected values (weighted average)
    for particle in particles:
        position = None
        angle = None

        expected_value_position += None
        expected_value_direction += np.vstack([np.cos(angle), np.sin(angle)]) * particle.weight

    # get the angle value from the averaged directions
    expected_angle = np.atan2(expected_value_direction[1], expected_value_direction[0]) 
    expected_angle = np.vstack([expected_angle])

    # Calculate the expected value of the robot pose
    expected_pose = np.concatenate([expected_value_position, expected_angle], 0)

    # Calculate the diagonal elements of the covariance matrix (we will ignore the rest to avoid dealing with angular differences)
    var_x = 0.
    var_y = 0.
    for particle in particles:
        var_x += None
        var_y += None

    # circular variance is inversely proportional to the length of the circular mean vector
    angular_variance = 1 - np.linalg.norm(expected_value_direction)
    covariance = np.diag([var_x, var_y, angular_variance])
    return expected_pose, covariance


Let's use this, then. We'll create a few random particles and evaluate each one. Then, we will look at those evaluations to extract the expected value and covariance of the estimated pose of the robot.

In [ ]:
def test_estimate_pose():
    true_pose =  np.vstack([4,4,0])
    laser = Laser2D(np.pi, 0.1, 10, np.diag([0.01, 0.0]), true_pose)
    z = laser.take_observation(virtual_map)
    
    num_particles = 300
    particles = []
    for i in range(num_particles):
        # uniformly distributed random particles in the general vecinity of the real pose
        pose_particle = np.vstack([
            (np.random.rand()-0.5) * 2 + true_pose[0],
            (np.random.rand()-0.5) * 2 + true_pose[1],
            (np.random.rand()-0.5) * 1 + true_pose[2],
        ])
        # get the evaluation for this random pose and add it to the list
        weight = evaluate_pointcloud(occupancy_map, LaserUtils.express_as_global_point_cloud(z, pose_particle))
        particles.append(Particle(pose_particle, weight))

    # use the particles and their evaluations to estimate the real pose
    estimated_pose, estimated_cov = current_estimation(particles)
    
    # plot
    fig, ax = occupancy_map.show_likelihood()
    DrawRobot(fig, ax, true_pose, 0.01, linewidth=2, color="red")
    DrawRobot(fig, ax, estimated_pose, 0.01, linewidth=2, color="green")
    PlotEllipse(fig, ax, estimated_pose[0:2], estimated_cov, color="limegreen")

test_estimate_pose()

<span style="color:blue">Expected output:</span>

<img src="images/likelihood_field_with_estimated_pose.png">

We can see the expected value does not always give us a perfect estimation, but it tends to be quite close. If we increase the number of particles, the estimations tend to be more reliable and the uncertainty is lower. Try different numbers of particles and check this for yourself!

### Sampling and Resampling

We know how to evaluate particles, and how to generate a final estimation from those particles. However, where do those particles come from? Well, initially, they are just generated around the starting pose of the robot. We can do this by just taking samples of the pose's probability distribution, like we did all the way at the beginning:

In [ ]:
def generate_particles(num_particles, pose, covariance):
    particles : list[Particle] = []
    for i in range(num_particles):
        particle_pose = np.vstack(stats.multivariate_normal.rvs(pose.flatten(), covariance))
        particles.append(Particle(particle_pose, 1. / num_particles)) # equal weight to start with, we don't yet know how good these particles are
    return particles

After that, there are two basic operations to generate particles: **sampling** (using the previous generation of particles and the control action), and **resampling** (just trying new random values). Let's see them.

**Sampling** is easy enough: we take each existing particle from the previous iteration $x_{t-1}$, and we move it according to the movement $\hat{u_t}$ that the robot tried to do. The key is that, since the movement has uncertainty, the particle actually receives a noisy movement, which doesn't perfectly correspond to the control action. This noisy movement is generated by using the covariance of the movement, $\Sigma_u$, to sample its probability distribution. The final pose of the particle is then $$ x_t = x_{t-1} \oplus u_t$$ , with $$u_t \sim N(\hat{u_t}, \Sigma_u)$$

### **<span style="color:green"><b><i>ASSIGNMENT 3: Sampling particles</i></b></span>**  

Complete the `sample()` function below, which takes a list of particles (representing hypotheses for the pose of the robot) and a movement command. The function then modifies each the particles to estimate the pose of the robot after making the movement.

You need to also complete the `generate_random_movement()` function, which takes a movement command, expressed as a pose increment $u = [\Delta x, \Delta y, \Delta \theta]^T$ and its covariance, and generates a random movement from it. The random movement is generated using the sampling model discussed in notebook 3.3, where the movement is expressed as $u = [\theta_1, d, \theta_2]^T$:

1. **Turn** ($\theta_1$): to face the destination point.
2. **Advance** ($d$): to arrive at the destination.
3. **Turn** ($\theta_2$): to get to the desired angle.

Remember that you can convert between the two representations with the following formulas:

\begin{split}
    & \theta_1 =atan2(\Delta y, \Delta x) \\
    & d = \sqrt{(\Delta x)^2 + (\Delta y)^2} \\
    & \theta_2  = \theta - \theta_1 
\end{split}

The jacobian for this conversion, which will be necessary to calculate this covariance, is:

\begin{bmatrix}
    \frac{- \Delta y}{d^2} &  \frac{\Delta x}{d^2} & 0 \\
    \frac{\Delta x}{d} &  \frac{\Delta y}{d} & 0 \\
    \frac{\Delta y}{d^2} &   \frac{- \Delta x}{d^2} & 1
\end{bmatrix}

In [ ]:
def generate_random_movement(ideal_movement, movement_covariance):
    x = ideal_movement[0]
    y = ideal_movement[1]
    theta = ideal_movement[2]

    # change the movement command to the angle-movement-angle mode to generate the random sample
    a1 = np.arctan2(y, x)
    d = np.sqrt(x*x + y*y)
    a2 = theta - a1
    mean_sampling = np.vstack([None, None, None])
    
    jacobian = np.array([
        [-y/(d**2), x/(d**2), 0],
        [x/d, y/d, 0],
        [y/(d**2), x/(d**2), 1]
    ])

    cov_sampling = None @ None @ None
    
    movement_sample = np.vstack(stats.multivariate_normal.rvs(None, None))
    
    # revert to pose increment so we can do the composition
    pose_increment_sample = np.vstack([
        movement_sample[1] * np.cos(movement_sample[0]),
        movement_sample[1] * np.sin(movement_sample[0]),
        movement_sample[0] + movement_sample[2]
    ])

    return pose_increment_sample


def sample(particles, ideal_movement, movement_covariance):
    for particle in particles:
        noisy_movement = generate_random_movement(ideal_movement, movement_covariance)
        particle.pose = tcomp(particle.pose, noisy_movement)

# function to plot the particles
def show_particles(particles, fig, ax, size = 0.002, color="cyan"):
    for particle in particles:
        DrawRobot(fig, ax, particle.pose, size, color=color)

Now test your code:

In [ ]:
def test_sample_function():
    fig, ax = occupancy_map.show_occupancy()

    ideal_movement = np.vstack([2, 0, 0])
    movement_covariance = np.diag([0.02, 0.04, 0.02])
    
    # initial state
    initial_pose = np.vstack([4,4,0])
    initial_pose_cov = np.diag([0.05, 0.05, 0.04])
    DrawRobot(fig, ax, initial_pose, linewidth=1.)
    particles = generate_particles(20, initial_pose, initial_pose_cov)
    show_particles(particles, fig, ax, size=0.007, color="orange")
    
    # simulate one movement
    sample(particles, ideal_movement, movement_covariance)
    show_particles(particles, fig, ax, size=0.007, color="cyan")

    # simulate another movement
    sample(particles, ideal_movement, movement_covariance)
    show_particles(particles, fig, ax, size=0.007, color="limegreen")

test_sample_function()

<span style="color:blue">Expected output:</span>

<img src="images/particle_filter_sampling.png">

After doing the sampling step, we might decide that some of these particles are very bad, and thus not interesting. Rather than keep propagating these bad estimations, we would prefer to allocate our computational resources to evaluating candidate poses that are more likely to be close to the correct value. *That* is the idea behind **resampling**. We eliminate particles that got a low evaluation, and generate new particles to replace them. 

These new particles can be generated many different ways, but it is generally a good idea to use the current estimation of the pose (or the particles which got a good evaluation) as a reference value around which to spawn the new particles. This is a technique known as [importance sampling](https://en.wikipedia.org/wiki/Importance_sampling), and it allows us to get a good estimation even with a relatively small number of particles, since we avoid wasting resources by creating particles that are very far away from the pose we already know is likely to be correct.

### **<span style="color:green"><b><i>ASSIGNMENT 4: Resampling particles</i></b></span>**  

Let's do just that, then. We will use the provided function `eliminate_bad_particles()` to get rid of the particles whose evaluation is below a certain threshold. Then, we'll use the `resample()` function, which you must complete, to generate new particles. 

The new particles must spawn around the current estimated pose, using the uncertainty of the estimation to draw the new samples. A new particle $i$ would then be generated by drawing a sample pose $x_i$:

$$
x_i \sim N(E[X], \Sigma)
$$

Optionally, one could "scale up" the uncertainty by an arbitrary factor $\lambda$, to make sure the new particles generated cover a wider spectrum of hypothetical poses. This way, we can avoid having all the new particles be very similar to the old ones:

$$
x_i \sim N(E[X], \lambda \cdot \Sigma)
$$


In [ ]:
def eliminate_bad_particles(particles, kill_weight_threshold):
    # the simplest possible approach to eliminating particles: check their likelihood against a fixed threshold
    is_good = lambda p : p.weight > kill_weight_threshold 
    return [p for p in particles if is_good(p)]

# generate new particles around the current estimation for the robot pose
# the more uncertainty we have, the more area we want the new particles to cover
def resample(particles, desired_particle_count, map):  
    SPAWN_RADIUS_MULTIPLIER = 2 # arbitrarily make the sampling area bigger to avoid getting stuck on a bad estimation 
    estimated_pose, estimated_covariance = current_estimation(particles)
    sigma_x = np.sqrt(estimated_covariance[0,0]) * SPAWN_RADIUS_MULTIPLIER
    sigma_y = np.sqrt(estimated_covariance[1,1]) * SPAWN_RADIUS_MULTIPLIER
    sigma_angle = np.sqrt(estimated_covariance[2,2]) * SPAWN_RADIUS_MULTIPLIER

    while len(particles) < desired_particle_count:
        pose = np.vstack([
            None
        ])

        if map.is_valid_position(pose[0:2]):
            particles.append(None)
    
    # Technically, since we are doing a form of importance sampling (https://en.wikipedia.org/wiki/Importance_sampling),
    # the resulting weights should not be uniform...
    # but then, this is a simple example implementation :)
    for particle in particles:
        particle.weight = 1. / len(particles)

Test the resampling with the following code. The particles that are eliminated will be shown in grey, the particles that survive will be shown in orange, and the newly sampled particles will be shown in cyan.

In [ ]:
def test_resampling():
    fig, ax = occupancy_map.show_occupancy()

    movement = np.vstack([2, 0, 0])
    movement_covariance = np.diag([0.02, 0.04, 0.02])
    
    # initial state
    initial_pose = np.vstack([4,4,0])
    initial_pose_cov = np.diag([0.05, 0.05, 0.04])
    particles = generate_particles(20, initial_pose, initial_pose_cov)
    DrawRobot(fig, ax, initial_pose, linewidth=1.)

    # move and sample
    real_pose = tcomp(initial_pose, movement)
    sample(particles, movement, movement_covariance)
    DrawRobot(fig, ax, real_pose, linewidth=1., color="orange")
    show_particles(particles, fig, ax, size=0.007, color="grey")
    
    # evaluate
    laser = Laser2D(np.pi, 0.1, 10, np.diag([0.01, 0.0]), real_pose)
    z = laser.take_observation(virtual_map)
    for particle in particles:
        particle.weight = evaluate_pointcloud(occupancy_map, LaserUtils.express_as_global_point_cloud(z, particle.pose))
    normalize_weigths(particles)

    # resample
    particles = eliminate_bad_particles(particles, 1e-5)
    show_particles(particles, fig, ax, size=0.007, color="orange")

    display(fig)        
    clear_output(wait=True)
    time.sleep(1.0)

    old_particles = copy.deepcopy(particles)
    resample(particles, 20, occupancy_map)
    show_particles(particles, fig, ax, size=0.007, color="cyan")
    show_particles(old_particles, fig, ax, size=0.007, color="orange")
    display(fig)        
    clear_output(wait=True)


test_resampling()
    

<span style="color:blue">Expected output:</span>

<img src="images/particle_filter_resampling.png">

## 5.3.3 Assembling the Particle Filter

We now have all the pieces of the particle filter, so let's put them in order and see the algorithm come to life.

### **<span style="color:green"><b><i>ASSIGNMENT 5: Putting all together</i></b></span>**  

First, complete the body of the function`update_weights()` in the `ParticleFilter` class below. Remember that the weight of a particle at time instant $t$ is updated recursively based on the likelihood estimated for the most recent measurement (which we calculated in the evaluation section):

$$
w^i_t \propto w^i_{t-1} \cdot p(z^w_t | m)
$$


In [ ]:
class ParticleFilter:
    def __init__(self, initial_pose):
        # You can play with these parameters
        self.min_particles = 30
        self.max_particles = 70
        self.kill_weight_threshold = 1e-3

        num_particles_init = 50
        self.particles = generate_particles(num_particles_init, initial_pose, np.diag([0,0,0]))

    def update_weights(self, map, z):
        for particle in self.particles:
            None # update the particle weight using the observation

        normalize_weigths(self.particles)


    def refresh_particles(self, map):
        self.particles = eliminate_bad_particles(self.particles, self.kill_weight_threshold)

        # In a real implementation we would check a degeneracy metric rather than look at the particle count
        if len(self.particles) < self.min_particles:
            resample(self.particles, self.max_particles, map)

Now, let's run the particle filter algorithm! Complete the main loop below to estimate the pose of the robot as it moves around the environment. You will need to use the functions that we have defined previously in the notebook for each of the steps.

The core process is as follows:

- Apply the movement command to all particles (sampling).
- Evaluate each candidate pose given the most recent measurement.
- Calculate the estimated pose and its associated uncertainty.
- Eliminate bad particles and (maybe) generate new ones (resampling).

In [ ]:
def test_particle_filter():
    true_pose = np.vstack([4,4,0])
    ideal_pose = true_pose 

    movements = [
        np.vstack([2, 0, 0]),
        np.vstack([2, 0, 0]),
        np.vstack([0, 2, np.pi/2]),
        np.vstack([2, 0, 0]),
        np.vstack([0, 2, np.pi/2]),
        np.vstack([2, 0, 0]),
    ]
    cov = np.diag([0.02, 0.02, 0.01])

    laser = Laser2D(np.pi, 0.1, 10, np.diag([0.01, 0.0]), true_pose)
    pf = ParticleFilter(true_pose)

    fig, ax = occupancy_map.show_occupancy()
    DrawRobot(fig, ax, ideal_pose, 0.015, linewidth=1.5, color="red")
    plt.ion()

    old_particles=[]

    for move in movements:
        # move and take a reading
        ideal_pose = tcomp(ideal_pose, move)
        DrawRobot(fig, ax, ideal_pose, 0.015, linewidth=1.5, color="red")

        true_pose = tcomp(true_pose, generate_random_movement(move, cov))
        DrawRobot(fig, ax, true_pose, 0.015, linewidth=1.5, color="blue")
        
        # take measurement
        laser.set_pose(true_pose)
        z = laser.take_observation(virtual_map)
        
        # MAIN EVENT: run the particle filter!
        #-------------------------
        sample(pf.particles, move, cov)
        pf.update_weights(occupancy_map, z)
        estimated_pose, estimated_covariance = current_estimation(pf.particles)
        pf.refresh_particles(occupancy_map)
        #-------------------------

        # plot the estimation
        DrawRobot(fig, ax, estimated_pose, 0.015, linewidth=1.5, color="green")
        # PlotEllipse(fig, ax, estimated_pose[0:2], estimated_covariance, color="green")

        # plot the active particles, and make the old ones a different color
        show_particles(pf.particles, fig, ax, color = "cyan")
        show_particles(old_particles, fig, ax, color = "lightgrey")
        old_particles = copy.deepcopy(pf.particles)

        # keep updating the plot on the same figure
        display(fig)        
        clear_output(wait=True)
        time.sleep(0.1)


test_particle_filter()

<span style="color:blue">Expected output:</span>

<img src="images/particle_filter_localization.png">

So that's the algorithm! As you can see, the process is conceptually much simpler than something like EKF, given we are always operating on specific poses rather than probability distributions, but there is a price to pay in terms of computational complexity and accuracy.

Still, Monte Carlo methods are very powerful tools, and very commonly used. Their ability to represent any type of distribution (rather than being constrained to gaussian approximations) means they are very flexible. The implementation we have explored here is a relatively simplistic one, but making only a few changes you could represent multi-modal distributions, consider whether specific trajectories would collide with obstacles, *etc.* 

## <font color="orange"><b><i>AI Appendix</i></b></font>

### A.1 How I used AI
- **Model/tool:**
  <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>
- **What I asked AI to help with (bullets):**
  <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>
- **Best prompt I used (paste):**
  <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>
- **What I kept vs. changed from the AI output:**
  <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>

### A.2 AI review 
Ask one generative AI to review your memo using this prompt:

```
Act as a technical reviewer of this computer vision lab notebook. Read my memo and return exactly 5 bullet points:

- two on theory (key ideas),
- two on practice (relevant programming concepts of python used, not particular functions or variables),
- one on how to improve the code (most critical aspect: limitations, parameter choices, runtime/robustness, possible bugs).

Keep each bullet ≤15 words and make them specific to my work.
```

Paste its **5** items here:

- **(Theory 1)** AI claim:
  <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>
- **(Theory 2)** AI claim:
  <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>
- **(Practice 1)** AI claim:
  <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>
- **(Practice 2)** AI claim:
  <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>
- **(Improvement)** AI claim:
  <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>

Finally, **do you agree with them?**, **would you have selected others?** 
    <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>